In [1]:
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn

This training would be done on the coding language Java

In [2]:
with open('Datasets/text/javaer.txt', 'r') as f:
    data = f.read()
f.close()

In [3]:
# now we have the data
vocab = sorted(list(set(data)))
voc_sz = len(vocab)  #--
data_sz = len(data)  #--

itoa = {}
atoi = {}
for i,ele in enumerate(vocab):
    atoi[ele] = i
    itoa[i] = ele

In [4]:
train = np.zeros([data_sz,1])
output = np.zeros([data_sz,1])
for i in range(data_sz):
    train[i] = atoi[data[i]]
    if i!=0:
        output[i-1] = atoi[data[i]]


In [5]:
train.shape

(4000000, 1)

An RNN layer is a repeating neural net architecture. Basically it cycles through a set of input tokens (lets say an example of 8 token sequence) while keeping the previous token's activation (memory kind of) continued to the next token.

Maintaining the activations of the previous token helps to enhance some aspects of the upcoming tokens and continue to establish the activation mapping of the entire sequence

It can be thought of a compressed mapping of a sequence 

In [6]:
# lets define an RNN layer

class layer:
    def __init__(self,Wax:torch.tensor,Waa:torch.tensor,Wy:torch.tensor,ba:torch.tensor):
        self.Wax = Wax
        self.Waa = Waa
        self.Wy = Wy
        self.ba = ba
        self = self
        pass

    def forward(self,A:torch.tensor,X:torch.tensor):

        X_inp = torch.tensor((X,voc_sz))
        A_act = torch.tanh((self.Wax @ X_inp) + self.Waa @ A + self.ba)
        Y_act = (self.Wy @ A_act)
        logits = torch.tanh(Y_act)
        logits.requires_grad = True
        return A_act,logits

    def optimise(self,logit:torch.tensor,Y:torch.tensor):
        y = torch.tensor(Y)
        loss = F.cross_entropy(logit,y)
        opt = torch.optim.SGD(params=[self.Wax,self.Waa,self.Wy,self.ba,self.by],lr=0.1)
        #opt.zero_grad()
        loss.backward()
        opt.step()
        return loss


In [7]:
# -------------------------------- definitions ---------------------------------
n_hid = 64
epochs = 3
batch = 200
iteration = int(data_sz/batch)

Wax = torch.rand(n_hid, voc_sz) *0.01
Waa = torch.rand(n_hid, n_hid) *0.01
Wy = torch.rand(voc_sz, n_hid)
# ba = torch.zeros(n_hid,1)

Wax.requires_grad = True
Waa.requires_grad = True
Wy.requires_grad = True
# ba.requires_grad = True

parameters = [Wax,Waa,Wy]
# opt = torch.optim.SGD([Wax,Waa,Wy],lr=0.1)

#-------------------------------------------------------------------------------


In [36]:
lr = 1000

In [37]:
optimizer = torch.optim.SGD(parameters, lr=lr)

The activations are reset every new sequence (Training example) but the weights and biases are the matrices that are ultimately trained

In [41]:
# learning loop
#A_new.requires_grad = True

# ___ The traaining should be done much more than 10 ___
for x in range(5): # data_sz-batch):
    ix = random.randint(0, data_sz)
    A_old  = torch.zeros(n_hid,1)
    OLP = Wax.clone().detach()
    #print(Wax.grad)
    #opt.zero_grad()
    optimizer.zero_grad()
    for i in range(batch):
        
        # ---------------------------- inpt/outp -------------------------------
        X_input = torch.tensor(train[ix+i]).view([1,1]).long()
        X_inp = torch.tensor(F.one_hot(X_input,voc_sz)).view([voc_sz,1])
        Y = torch.tensor(output[ix+i]).long()
        # ----------------------------------------------------------------------


        # ----------------------------- forward --------------------------------
        A_new = torch.tanh((Wax @ X_inp.float()) + Waa @ A_old)*0.3
        Y_act = torch.relu(Wy @ A_new)
        logits = F.softmax(Y_act.view([voc_sz])).view([1,voc_sz])
        probs = torch.exp(logits)/torch.sum(torch.exp(logits[0]))
        # ----------------------------------------------------------------------
        # print(probs)

        # ----------------------------- Backprop -------------------------------
        loss = F.cross_entropy(probs,Y)
        loss.backward()
        # ----------------------------------------------------------------------

        #-------- Memory maintain -------
        A_old = A_new.clone().detach()
        #--------------------------------
    optimizer.step()
    # Wax -= lr*Wax.grad   
    # Waa -= lr*Waa.grad  
    # Wy -= lr*Wy.grad
    print("loss: ",loss.item())
    

    # break
    #print(Waa ,Waa.grad)
    # print(Wax,"\n",Waa,"\n",Wy)




C:\Users\soumi\AppData\Local\Temp\ipykernel_13272\3347931127.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_inp = torch.tensor(F.one_hot(X_input,voc_sz)).view([voc_sz,1])
C:\Users\soumi\AppData\Local\Temp\ipykernel_13272\3347931127.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(Y_act.view([voc_sz])).view([1,voc_sz])


loss:  4.543487548828125
loss:  4.543487548828125
loss:  4.525536060333252
loss:  4.543487548828125
loss:  4.525536060333252


/\ This is not the entire training 

When we say optimization this is what is happening under the hood :
 
Wax -= (lr) x (Wax.grad)   
Waa -= (lr) x (Waa.grad)  
Wy -= (lr) x (Wy.grad)

In [23]:
def forward(arg:str):
    X_input = torch.tensor(atoi[arg]).view([1,1]).long()
    X_inp = torch.tensor(F.one_hot(X_input,voc_sz)).view([voc_sz,1])
    A_pe = torch.tanh((Wax @ X_inp.float()) + Waa @ A_new)
    Y_act = (Wy @ A_pe)
    logits = torch.tanh(Y_act)
    probs = F.softmax(logits.view([voc_sz]))
    return probs
def predict(arg:str,lent:int):
    output = ""
    for i in range(lent):
        probs = forward(arg)
        ix = torch.multinomial(probs, num_samples=1).item()
        output+=itoa[ix]
    return output



In [101]:
arr = predict(" ",100)
print("-------------------------------------------\n")
print(arr)
print("\n--------------------------------------------")
print("\n\n\n")

-------------------------------------------

l>|,sD f9mk+\l:i,w=ZO	&Rwc^Q@./bz8u	/#_
.+
*lf[}|}Xbm
>V9L1>9r|_"y$lluljMl5pC+C|
<Q3YT>	9|aN{Q-Zgx S

--------------------------------------------






C:\Users\soumi\AppData\Local\Temp\ipykernel_13272\876156986.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_inp = torch.tensor(F.one_hot(X_input,voc_sz)).view([voc_sz,1])
C:\Users\soumi\AppData\Local\Temp\ipykernel_13272\876156986.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(logits.view([voc_sz]))


This is not an ideal output or training it was just to signify how an RNN works 
I hope this helps